In [1]:
import pandas as pd
import yahoo_fin.stock_info as si
import numpy as np

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
ETF_list=['FENY', 'VAW', 'FIDU', 'VCR', 'FSTA', 'FHLC', 'FNCL', 'XSW', 'VOX', 'FUTY', 'USRT', 'LQD']
portfolio = pd.DataFrame()

In [3]:
for stk in ETF_list:
    st_data = si.get_data(stk, start_date = "11/20/2016", end_date = "11/20/2020")
    portfolio[stk] = st_data['adjclose']
    
print(portfolio.iloc[10:])

                 FENY         VAW       FIDU         VCR       FSTA  \
2016-12-06  17.902319  106.234299  30.364428  122.166023  27.469191   
2016-12-07  18.003651  107.634201  30.848616  124.303871  27.873552   
2016-12-08  18.104980  108.552010  30.746185  124.660156  27.783695   
2016-12-09  18.172535  108.366585  30.876547  124.866432  28.188047   
2016-12-12  18.282318  107.662003  30.764814  123.703751  28.295876   
2016-12-13  18.501875  107.420982  30.653076  124.650787  28.394718   
2016-12-14  18.079647  106.174583  30.308554  123.841942  28.116167   
2016-12-15  18.147202  106.919807  30.355112  123.502579  28.259933   
2016-12-16  18.247353  106.398163  30.313038  123.116096  28.386578   
2016-12-19  18.170969  106.323639  30.462645  123.332901  28.431805   
2016-12-20  18.145508  106.612404  30.658993  124.417007  28.386578   
2016-12-21  18.170969  106.649658  30.584188  124.369850  28.413719   
2016-12-22  18.247353  105.867188  30.500038  122.936966  28.377533   
2016-1

In [4]:
ptf_logrtn = portfolio.apply(np.log).diff()
ptf_logrtn.head()

,FENY,VAW,FIDU,VCR,FSTA,FHLC,FNCL,XSW,VOX,FUTY,USRT,LQD
2016-11-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-11-22,0.000490,0.009245,0.006570,0.013052,0.005538,-0.013229,0.003033,0.000892,0.014807,0.005236,0.016706,0.002567
2016-11-23,0.003912,0.003834,0.008075,0.001917,-0.005865,0.005433,0.005436,0.002315,0.006640,-0.010499,-0.004684,-0.003682
2016-11-25,-0.004402,0.002489,0.004629,0.002753,0.006515,0.003906,0.001806,0.006383,0.005969,0.014408,0.005108,0.000086
2016-11-28,-0.015313,-0.005073,-0.007417,-0.007897,0.002918,-0.009339,-0.012708,-0.009055,0.005000,0.018361,0.005294,0.004535


In [5]:
cov_matrix_annual = ptf_logrtn.cov() * 252
print(cov_matrix_annual)

          FENY       VAW      FIDU       VCR      FSTA      FHLC      FNCL  \
FENY  0.114650  0.062521  0.063227  0.050382  0.030316  0.040038  0.071162   
VAW   0.062521  0.056661  0.052035  0.043828  0.028634  0.035793  0.055084   
FIDU  0.063227  0.052035  0.055888  0.044462  0.028897  0.036382  0.056955   
VCR   0.050382  0.043828  0.044462  0.048182  0.025635  0.033903  0.046361   
FSTA  0.030316  0.028634  0.028897  0.025635  0.028610  0.024920  0.030430   
FHLC  0.040038  0.035793  0.036382  0.033903  0.024920  0.038111  0.037995   
FNCL  0.071162  0.055084  0.056955  0.046361  0.030430  0.037995  0.070758   
XSW   0.048989  0.043373  0.043990  0.046251  0.024569  0.036519  0.044996   
VOX   0.045216  0.038329  0.038891  0.040534  0.025321  0.031990  0.041396   
FUTY  0.034609  0.031918  0.032277  0.026114  0.028915  0.027195  0.034826   
USRT  0.054330  0.044227  0.045510  0.039663  0.030698  0.032616  0.050183   
LQD   0.006698  0.004948  0.005413  0.005814  0.003886  0.003731

In [6]:
weights = np.array([0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.2])
ptf_annual_rtn = np.sum(ptf_logrtn.mean()*weights) * 252
print('portfolio annual return: ', ptf_annual_rtn)
ptf_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
ptf_volatility = np.sqrt(ptf_variance)
print('portfolio volatility/ standard deviation: ',ptf_volatility)
print('portfolio variance: ', ptf_variance)

portfolio annual return:  0.08674231459211953
portfolio volatility/ standard deviation:  0.16875414105533182
portfolio variance:  0.028477960123322824


In [7]:
from scipy.optimize import minimize

In [8]:
# minimize negative Sharpe Ratio
def neg_sharpe(weights): 
    ptf_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
    ptf_volatility = np.sqrt(ptf_variance)
    return ptf_volatility

# check allocation sums to 1
def check_sum(weights): 
    return np.sum(weights) - 1

cons = ({'type':'eq','fun':check_sum})

# create weight boundaries (-1 to 1 for the first weight to have a short position for the first ETF stock)
bounds = ((-1,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0.2,0.2))

# initial guess
init_guess = [0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.07272727, 0.2]

opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)

print(opt_results.x)

ptf_df = pd.DataFrame()
cnst_rtn = 0.05

ptf_variance_1 = np.dot(opt_results.x.T, np.dot(cov_matrix_annual, opt_results.x))
ptf_volatility_1 = np.sqrt(ptf_variance_1)
ptf_rtn_1=np.sum(ptf_logrtn.mean()*opt_results.x) * 252
sharpe_ratio=(ptf_rtn_1-cnst_rtn)/ptf_volatility_1


print(ptf_volatility_1, ptf_rtn_1, sharpe_ratio)

[-6.91204676e-02  0.00000000e+00  1.32814766e-18  6.61905763e-03
  6.12739566e-01  1.87611234e-01  1.08420217e-18  1.32280333e-03
  6.08278066e-02  0.00000000e+00  0.00000000e+00  2.00000000e-01]
0.137968703162674 0.1151769625437592 0.4724039658973336


13.7% risk, 11.5% returns and 47% sharpe ratio